In [17]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import joblib

In [18]:
data = pd.read_csv("T1.csv")
data.head()

,Date/Time,LV ActivePower (kW),Wind Speed (m/s),Theoretical_Power_Curve (KWh),Wind Direction (°)
0,01 01 2018 00:00,380.047791,5.311336,416.328908,259.994904
1,01 01 2018 00:10,453.769196,5.672167,519.917511,268.641113
2,01 01 2018 00:20,306.376587,5.216037,390.900016,272.564789
3,01 01 2018 00:30,419.645905,5.659674,516.127569,271.258087
4,01 01 2018 00:40,380.650696,5.577941,491.702972,265.674286


In [19]:
# Handle missing values
data.fillna(method='ffill', inplace=True)

In [20]:
# Convert Date/Time to datetime
data['Date/Time'] = pd.to_datetime(data['Date/Time'], format='%d %m %Y %H:%M')


In [21]:
# Extract time-based features
data['year'] = data['Date/Time'].dt.year
data['month'] = data['Date/Time'].dt.month
data['day'] = data['Date/Time'].dt.day
data['hour'] = data['Date/Time'].dt.hour


In [22]:
# Drop the original Date/Time column
data = data.drop(['Date/Time'], axis=1)

In [23]:
# Define features and target variable
X = data.drop(['LV ActivePower (kW)'], axis=1)
y = data['LV ActivePower (kW)']

In [24]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [25]:
# Scale numerical features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [26]:
# Define the model
model = RandomForestRegressor(n_estimators=100, random_state=42)

In [27]:
# Train the model
model.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [28]:
# Make predictions
y_pred = model.predict(X_test)

In [29]:
# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test,y_pred)
print(f'Mean Absolute Error: {mae}')
print(f'Mean Squared Error: {mse}')
print(f'R Squared: {r2}')

Mean Absolute Error: 78.67278301897909
Mean Squared Error: 46371.06305642386
R Squared: 0.9728238043410564


In [30]:
# Save the model
joblib.dump(model, 'wind_turbine_scada_model.pkl')
joblib.dump(scaler, 'scaler.pkl')


['scaler.pkl']

In [31]:
# Example new data (replace with actual new data)
new_data = pd.DataFrame({
    'Wind Speed (m/s)': [12.512312213],
    'Theoretical_Power_Curve (KWh)': [3500.21312313],
    'Wind Direction (°)': [180.12312323],
    'year': [2080],
    'month': [7],
    'day': [2],
    'hour': [15]
})

In [32]:
# Load the scaler
scaler = joblib.load('scaler.pkl')

# Scale the new data using the previously fitted scaler
new_data_scaled = scaler.transform(new_data)

# Load the model
model = joblib.load('wind_turbine_scada_model.pkl')

# Predict using the trained model
new_predictions = model.predict(new_data_scaled)
print(f'Predicted LV Active Power (kW): {new_predictions}')

Predicted LV Active Power (kW): [2907.38802246]
